In [40]:
import re
def cleaningLinePAN(aLine):
   if (aLine.find("<author") >= 0) :
      aLine = ""
   if (aLine.find("<conversation") >= 0) :
      aLine = ""
   if (aLine.find("</conversation") >= 0) :
      aLine = ""
   if (aLine.find("</author") >= 0) :
      aLine = ""
   if (aLine.find("<documents>") >= 0) :
      aLine = ""
   if (aLine.find("</documents>") >= 0) :
      aLine = ""
   aLine = aLine.replace("</document>", "")
   aLine = aLine.replace("\n", " ")
   aLine = aLine.replace("&quot;", '"')
   aLine = aLine.replace("&gt;", ">")
   aLine = aLine.replace("&lt;", "<")
   aLine = aLine.replace("&nbsp;", " ")
   aLine = aLine.replace("&amp;", "&")
   aLine = aLine.replace("<>", " ")
   aLine = aLine.replace("[...]", "")
   aLine = aLine.replace("<![CDATA[", "")
   aLine = aLine.replace("]]>", "")
   aLine = aLine.replace("  ", " ")
   if (len(aLine) > 1):
      aLine = aLine.replace("\t"," ")
   if (len(aLine) < 1):
      return("")

   aMatch = re.search("<br[ ]?[/]?>", aLine)
   while (aMatch):
      aLine = aLine[:aMatch.start()] + " " + aLine[aMatch.end():]
      aMatch = re.search("<br[ ]?[/]?>", aLine)
      
   aMatch = re.search("<a .*?>", aLine)
   while (aMatch):
      aLine = aLine[:aMatch.start()]+ " AREF " + aLine[aMatch.end():]
      aMatch = re.search("<a[.]*>", aLine)
      
   aMatch = re.search("<.*?>", aLine)
   while (aMatch):
#     print "<TAGS", aMatch.group(0), aMatch.start(), aMatch.end()
      aLine = aLine[:aMatch.start()]+ " " + aLine[aMatch.end():]
      aMatch = re.search("<.*?>", aLine)
         
   aMatch = re.search("\&.*?;", aLine)
   while (aMatch):
#      print "<CODE", aMatch.group(0), aMatch.start(), aMatch.end()
      aLine = aLine[:aMatch.start()] + " " + aLine[aMatch.end():]
      aMatch = re.search("\&.*?;", aLine)
# Coded character:  If the string  aLine is in UTF-8 nothing is modified     
   aMatch = re.search(r"\\u\d{3,4}", aLine, re.I)
   while (aMatch):
#      print "Code U" , aMatch.group(0), aMatch.start(), aMatch.end()
      aLine = aLine[:aMatch.start()-1] + " " + aLine[aMatch.end():]
      aMatch = re.search("\\u\d{3,4}", aLine, re.I)

# http://anURL    
   aMatch = re.search("http[s]?://[A-Za-z0-9/\.]* ?", aLine, re.I)
   while (aMatch):
#    print "http" , aMatch.group(0), aMatch.start(), aMatch.end()
      aPos = max(aMatch.start()-1, 0)
      aLine = aLine[:aPos] + " urllink " + aLine[aMatch.end():]
      aMatch = re.search("http[s]?://[A-Za-z0-9/\.]* ?", aLine)
      
   aLine = aLine.replace("  ", " ")
   if (aLine == " "):
      aLine = ""
   return(aLine)






In [41]:
import unicodedata
import sys
if sys.version_info[0] >= 3:
          unicode = str
#
# Code  L = letter
#       Cc = control char (e.g.. newline)
#       Cs = emoticons
#       Nd = numeric
#       P. = punctuation
#       Zs  = space
#       Sc  = symbol (e.g. $)
#       Sm  = symbol (e.g. <)
#      


def tokenizeLineUTF8(newLine):
   aListChar = []
   aListCode = []
   aListCode2 = []
   for i, c in enumerate(newLine):
      aListCode.append(unicodedata.category(c)[0])
      aListCode2.append(unicodedata.category(c)[1])
      aListChar.append(c)
   seqChar = seqPunct = seqNumber = seqEmoji = seqSymbol = False
   aLen = len(aListCode)
   aToken = u''
   aListOfWord = []
   for anIndex in range(aLen):
      aCode  = aListCode[anIndex]
      aCode2 = aListCode2[anIndex]
      aChar = aListChar[anIndex]
      if ((aCode == "L") & (seqChar)):
         aToken = aToken + aChar
      elif ((aCode == "P") & (seqPunct | seqNumber)):
         aToken = aToken + aChar
         seqPunct = True
         seqNumber = False
      elif ((aCode == "N") & (seqPunct | seqNumber | seqSymbol)):
         aToken = aToken + aChar
         seqNumber = True
         seqPunct = seqSymbol = False
      elif ((aCode == "S") & (seqPunct | seqNumber | seqSymbol)):
         aToken = aToken + aChar
         seqSymbol = True
         seqPunct = seqNumber = False
      elif (((aCode == "C") & (aCode2 == "s")) & seqEmoji):
         aToken = aToken + aChar
      elif ((aCode == "Z") | ((aCode == "C") & (aCode2 != "s")) | (aCode == "M")):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = u''
         seqChar = seqPunct = seqNumber = seqEmoji = seqSymbol = False
      elif ((aCode == "L") & (not seqChar)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqChar = True
         seqPunct = seqNumber = seqEmoji = seqSymbol = False
      elif ((aCode == "P") & (not seqPunct)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqPunct = True
         seqChar = seqNumber = seqEmoji = seqSymbol = False
      elif ((aCode == "N") & (not seqNumber)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqNumber = True
         seqChar = seqPunct = seqEmoji = seqSymbol = False
      elif ((aCode == "S") & (not seqSymbol)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqSymbol = True
         seqChar = seqPunct = seqEmoji = seqNumber  = False
      elif (((aCode == "C") & (aCode2 == "s")) & (not seqEmoji)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqEmoji = True
         seqChar = seqPunct = seqNumber = seqSymbol = False
   if (len(aToken) > 0):
      aListOfWord.append(aToken)
   return(aListOfWord)


In [60]:
# -*- coding: utf-8 -*-
#
# Natural Language Toolkit: S-Stemmer
#
# Author: J. Savoy
# Algorithms: Donna Harman.  How effective is suffixing?
#      Journal of the American Society for Information Science
#      42(1), 1991, p. 7-15
# 
# URL: <http://nltk.org/>


"""
S-Stemmer
The S-Stemmer is a light English stemmer that is based on removing the suffixes
from the word related to the plural form.
This stemmer is not
based on any dictionary and can be used on-line effectively.
"""
from __future__ import unicode_literals
import re

from nltk.stem.api import StemmerI


class SStemmer(StemmerI):
    '''
    The S-Stemmer is a light English stemmer to remove the suffixes
    from plural nouns.
    '''

    def __init__(self):
        self.min = 0

    def stem(self, token):
        """
            call this function to get the word's stem based on S-Stemmer.
        """
        try:
            aLen = len(token)-1
            if (token is None) or (aLen < self.min):
                raise ValueError(
                    "The word could not be stemmed, because it is empty or too small !")
# Min word size = 4 letters            
            if (aLen <= 2):
                return token
# Must ends with -s
            if (token[aLen] != u's')and (len(token)>4):
                return token
# Rule 1  ending in -ies
            if (token[aLen-1] == u'e') and (token[aLen-2] == u'i'):
                if ((token[aLen-3] != u'e') and (token[aLen-3] != u'a')):
                    return (token[:aLen-2]+u'y')
                else:
                    return token
# Rule 2  ending in -es
            if (token[aLen-1] == u'e')and (len(token)>4):
                if ((token[aLen-2] != u'a') and (token[aLen-2] != u'e') and (token[aLen-2] != u'o')):
                    return token[:aLen]
                else:
                    return token
# Rule 3 ending in -s
            if ((token[aLen-1] == u"'")):
                return token
            
            if ((token[aLen-1] != u'u') and (token[aLen-1] != u's')) and (token != ("basis")) and(len(token)>4):
                return token[:aLen]
            else:
                return token
        except ValueError as e:
            print(e)

    def __repr__(self):
        return '<SStemmer>'

##text = ['houses','babies','cars','apples', "Mangoes","this", "paul's","basis"]
##for word in text:
##    print((word),SStemmer().stem(word))


In [190]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
import itertools
from collections import Counter
##from readfile033 import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import time
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
#tt = ToktokTokenizer()#keeps the url as is...................................................
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
start = time. time()

B_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_BOT_train.2.txt"
H_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_HUMAN_train.2.txt"

B_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_BOT_dev.2.txt"
H_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_HUMAN_dev.2.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN_2019_txtfiles/PAN2019_tweet_male_train.2.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN_2019_txtfiles/PAN2019_tweet_female_train.2.txt"


def split_list(a_list):
        #function to split array
    half = len(a_list)/2
    print("half",half)
    return a_list[:half], a_list[half:]

def all_txt(B_txt_file,  H_txt_file):
    with open(B_txt_file, mode="r", encoding="utf-8") as f:
            n=100
##            [f[i:i+n] for i in range(0, len(f), n)]))
            txt_list = [(line[:-1]).split("\t")  for line in f]
##            txt_list= [txt_list[i:i+n] for i in range(0, len(txt_list), n)]
            bot_type= [line[2] for line in txt_list]
            
            bot_gender= [line[3] for line in txt_list]
           
            bot_content= [line[4] for line in txt_list]
##            print(bot_content[0])
            bot_content= [(bot_type[i],bot_gender[i], bot_content[i:i+n]) for i in range(0, len(bot_content), n)]
            print("bot_content....",len(bot_content))
##            bot_content= [("bot", (' '.join(a)).split()) for a in bot_content]
##            bot_content= [(' '.join(a)).split() for a in bot_content]
            
##            bot_dict = dict()
##            content=[]
##            for line in txt_list:
##                print(line[1])
##                if line[1] in bot_dict:
##                # append the new number to the existing array at this slot if they have the same id
##                    bot_dict[line[1]].append(line[3])
##                else:
##                # create a new array in this slot
##                    bot_dict[line[1]] = [line[3]]
##    bot_content= [("bot", (' '.join(a)).split()) for a in list(bot_dict.values())]
##    bot_content= [bot_content[i:i+n] for i in range(0, len(bot_content), n)]
   
    with open(H_txt_file, mode="r", encoding="utf-8") as f:
            n=100
            txt_list = [(line[:-1]).split("\t") for line in f]
##            txt_list= [txt_list[i:i+n] for i in range(0, len(txt_list), n)]
            human_type= [line[2] for line in txt_list]
            human_gender= [line[3] for line in txt_list]
            human_content= [line[4] for line in txt_list]
            human_content= [(human_type[i], human_gender[i],  human_content[i:i+n]) for i in range(0, len(human_content), n)]
            print("human_content....",len(human_content))
##            human_content= [(' '.join(a)).split() for a in human_content] [item for sublist in bot_content[i:i+n] for item in sublist]
##            human_dict = dict()
##            content=[]
##            for line in txt_list:
##                if line[1] in human_dict:
##                # append the new number to the existing array at this slot
##                    human_dict[line[1]].append(line[3])
##                else:
##                # create a new array in this slot
##                    human_dict[line[1]] = [line[3]]
##    human_content= [("human", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = bot_content + human_content
    return all_txt_per_person

def all_txt1(M_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            n=100
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_type= [line[2] for line in txt_list]
            male_gender= [line[3] for line in txt_list]
            male_content= [line[4] for line in txt_list]
            male_content= [(male_type[i], male_gender[i],   male_content[i:i+n] ) for i in range(0, len(male_content), n)]
##            male_content= [("human", (' '.join(a)).split()) for a in male_content]
##    with open(F_txt_file, mode="r", encoding="utf-8") as f:
##            txt_list = [(line[:-1]).split("\t") for line in f]
##            female_content= [(line[2], line[3]) for line in txt_list]
##    all_txt_per_person = male_content + female_content
    return male_content


"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):
## all_training_text = combined text from all authors, separated on per author basis
## word_list = list containing selected words(selected terms) to be used as a feature set ... the word list is saved in a csv/txt file which can be parsed as required

    #pos_list = ['NNP', 'PRP', 'DT', 'NN', 'CD', 'NNS', 'IN' , 'VB', 'PRP$']
    fvs_words = np.array([[author.count(word) for word in word_list]
                           for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words

def WordFeatures1(word_list, all_training_text):
    fvs_words = np.array([(all_training_text.count(word))/(len(all_training_text)) for word in word_list]).astype(np.float64)
    return fvs_words

"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
word_list11=[]
#txt_files =["D:/NLP/PAN2019/PAN2019/2019_selectterms_b_h/outputNewVersionbothuman2019CHI.txt"]
#txt_files =["D:/NLP/PAN2019/PAN2019/2019_selectterms_b_h/outputNewVersionbothuman2019WLLR.1M.txt", "D:/NLP/PAN2019/PAN2019/2019_selectterms_b_h/outputNewVersionbothuman2019WLLR.2F.txt"]
#txt_files =["D:/NLP/PAN2019/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_bot_nh_chosen_set.1.csv", "D:/NLP/PAN2019/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_human_chosen_set.1.csv"]
##x =(pd.read_excel('D:/NLP/PAN2019/PAN2019/PAN2019-Terms.xls', header=0, parse_cols = "C"))
##y =(pd.read_excel('D:/NLP/PAN2019/PAN2019/PAN2019-Terms.xls', header=0, parse_cols = "F"))
#word_list=list(x.Term) + list(y.Term)
##print( list(y.Term))
word_listf=[]
word_listB=[]
word_listH=[]
fntxt_files=["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/listFunctionWords.EN.txt"]
Btxt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/VocUniq.Bot.txt"]
Htxt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/VocUniq.Human.txt"]

txt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_bot_chosen_set.2.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv"]
txt_files1 =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv"]

##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2019/2019TermSelection/New folder/outputBotHuman2019IG.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019CHI.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019GR.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/2019TermSelection/outputBotHuman2019IG.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019GSS.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019OR.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019OR.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019PMI.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019PMI.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019RF.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019RF.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019WLLR.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019WLLR.2F.txt"]##
#txt_files1 =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2019/terms_associated_to_Bot_or_Human/AVocUniq.Human.txt","/Users/catherine/Desktop/NLP/PAN Datasets/PAN2019/terms_associated_to_Bot_or_Human/AVocUniq.Bot.txt"]
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

for txt_file in txt_files1:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list2 =  [r[0] for r in reader]
        print(txt_file, len(word_list2))
        word_list11 = word_list11 + word_list2
print("length of features used =", len(word_list1))

#features for preclassification......................................................
for txt_file in Btxt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_listBB =  ([(r.split(":"))[1] for r in f])
        non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
        word_listBB = [(items.translate(non_bmp_map))[:-1] for items in word_listBB]
        word_listB = word_listB + word_listBB
##        print(txt_file, len(word_listB),word_listB)
        
for txt_file in Htxt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_listHH =  ([(r.split(":"))[1] for r in f])
        non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
        word_listHH = [(items.translate(non_bmp_map))[:-1] for items in word_listHH]
        word_listH = word_listH + word_listHH
        
for txt_file in fntxt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        fn_word_list =  ([r[:-1] for r in f])


##
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        word_list2 =  ([r.strip() for r in f])[0:100]
##        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))
##for txt_file in txt_files1:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        word_list1 =  ([(r.split(":"))[1] for r in f])
 #       non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
  #      word_list1 = ([(x.translate(non_bmp_map))[:-1] for x in word_list1])
   #     print(txt_file, len(word_list1))
    #    word_list = word_list + word_list1#
##print("length of features used =", len(word_list), (word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
Training_text= all_txt(B_train_file,  H_train_file)
##all_training_text = ( [((all_txt(B_train_file,  H_train_file))[i][2])  for i in range(len((all_txt(B_train_file,  H_train_file))))])
all_training_text = ( [((Training_text)[i][2])  for i in range(len(Training_text))] )
all_training_text=[(' '.join(a)).split() for a in all_training_text]
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
#print("shape...(all_training_text)",(X_train).shape)
##y_train=np.array( [((all_txt(B_train_file,  H_train_file))[i][0])  for i in range(len((all_txt(B_train_file,  H_train_file))))])
y_train=np.array( [((Training_text)[i][0])  for i in range(len(Training_text))])

print("len(y_train)=", (y_train.shape))
y_train11=np.array( [((Training_text)[i][1])  for i in range(len(Training_text))])
print("len(X_train)=",(X_train.shape), "len(y_train11)=", len(y_train))


Test_text=all_txt(B_test_file,  H_test_file)
all_test_text = ( [((Test_text)[i][2])  for i in range(len(Test_text))])
all_test_text=[(' '.join(a)).split() for a in all_test_text]
print("len(all_test_text)",len(all_test_text))
#XX_test = WordFeatures(word_list, all_test_text)
yy_test = np.array( [((Test_text)[i][0])  for i in range(len(Test_text))])
print("len(y_test11)=",(yy_test.shape))
y_test11 = np.array( [((Test_text)[i][1])  for i in range(len(Test_text))])
print("len(y_test11=",len(y_test11), "len(y_test)=",len(yy_test))



"""
Parsing to the 2nd classifies............................................................
"""
Test_text1=all_txt1(H_train_file)
all_test_text1 = ( [((Test_text1)[i][2])  for i in range(len(Test_text1))])
all_test_text1=[(' '.join(a)).split() for a in all_test_text1]
print("len(all_test_text)",len(all_test_text1))
#X_train1 = WordFeatures(word_list11, all_test_text1)
y_train1 = np.array( [((Test_text1)[i][1])  for i in range(len(Test_text1))])
print("y_train1...",y_train1)



#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train11[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)


#..................................................................................

train_data_list_1 = []
j=0
for f in y_train1:
   
    train_data_list_1.append((f, y_train1[j], all_test_text1[j]))
    j += 1
# We have training data available as dictionary filename, category, data
train_data_1 = pd.DataFrame.from_records(train_data_list_1, columns=data_tags)



#..................................................................................

test_data_list = []
ii=0
for f in yy_test:
   
    test_data_list.append((f, y_test11[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path


/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_bot_chosen_set.2.csv 5602
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv 5828
length of features used = 11430
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv 5828
length of features used = 5828
bot_content.... 1440
human_content.... 1440
len(all_training_text) 2880
len(y_train)= (2880,)
len(X_train)= (2880, 11430) len(y_train11)= 2880
bot_content.... 620
human_content.... 620
len(all_test_text) 1240
len(y_test11)= (1240,)
len(y_test11= 1240 len(y_test)= 1240
len(all_test_text) 1440
y_train1... ['male' 'male' 'male' ... 'female' 'female' 'female']


In [1]:
#data_classes = ["bot", "human"]

#train_posts = data['tweet'][:len(data)]
train_posts = train_data['tweet']
#train_tags = data['gender'][:len(data)]
train_tags = train_data['type']

#train_tags= data['type'].apply(data_classes.index)

#train_files_names = data['gender'][:len(data)]


#test_posts = data['tweet'][:len(data)]
test_posts = test_data['tweet']
#test_tags = data['gender'][:len(data)]
test_tags = test_data['type']
test_tags1 = test_data['gender']

#test_tags= data['type'].apply(data_classes.index)
#test_files_names = data['gender'][:len(data)]


train_posts_1 = train_data_1['tweet']
#train_tags = data['gender'][:len(data)]
train_tags_1 = train_data_1['gender']


NameError: name 'train_data' is not defined

In [232]:
# :Tokenize and Prepare Vocabulary
num_labels = 2
vocab_size = 15000#len(word_list)#most common number of words will be then kept for use in the vector
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_train1 = tokenizer.texts_to_matrix(train_posts_1, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

# Save Tokenizer i.e. Vocabulary..............................................
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
    #pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


#with open('tokenizer.pickle', 'rb') as f:
#    X_y = pickle.load(f)
#    print(X_y)
    
    
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))

"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
encoder1 = LabelBinarizer()
encoder1.fit(train_tags_1)
y_train1 = encoder1.transform(train_tags_1)
y_train1 = np.hstack((y_train1, 1 - y_train1))
y_test1 = encoder1.transform(test_tags1)
y_test1 = np.hstack((y_test1, 1 - y_test1))


[('urllink', 193447), ('@', 165096), ('.', 151979), ('the', 139330), (',', 136277), (':', 126043), ('#', 123982), ('be', 120483), ('to', 98881), ('a', 92129), ('and', 74195), ('of', 70438), ('-', 70182), ("'", 69279), ('in', 60206), ('rt', 57659), ('i', 53494), ('you', 52775), ('…', 52703), ('for', 48572), ('it', 44969), ('s', 44360), ('!', 35394), ('on', 32484), ('have', 30211), ('that', 27399), ('–', 26821), ('’', 26558), ('this', 26505), ('with', 26130), ('_', 22500), ('t', 22382), ('?', 21198), ('job', 20932), ('we', 19970), ('at', 19473), ('"', 19339), ('your', 18863), ('do', 18272), ('�', 17590), ('my', 17402), ('/', 17275), ('(', 15911), ('get', 15897), ('can', 14431), ('not', 14185), ('as', 13449), ('will', 13172), ('engineer', 13088), ('about', 13074), ('by', 12821), ('what', 12722), ('from', 12533), ('an', 12470), ('&', 12396), ('all', 12329), ('our', 12311), ('but', 12192), ('out', 11936), ('...', 11858), ('so', 11817), (')', 11808), ('just', 11421), ('if', 11415), ('develop

In [193]:
#the number of layers, kernel size, vocabulary size..................
#each layer is learning different levels of abstraction..............
#Build Keras 1st Model and Fit.with 2 labels for type.................
from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

#Build Keras 1st Model and Fit..with 3 labels for gender......................
num_labels = 2
model1 = Sequential()
model1.add(Dense(512, input_shape=(vocab_size,)))
model1.add(Activation('relu'))
#model.add(Dropout(0.3))
model1.add(Dense(512))
model1.add(Activation('relu'))
#model.add(Dropout(0.3))
model1.add(Dense(num_labels))
model1.add(Activation('softmax'))
model1.summary()
 
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history1 = model1.fit(x_train1, y_train1,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

 



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 512)               7680512   
_________________________________________________________________
activation_70 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_74 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_71 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 2)                 1026      
_________________________________________________________________
activation_72 (Activation)   (None, 2)                 0         
Total params: 7,944,194
Trainable params: 7,944,194
Non-trainable params: 0
_________________________________________________________________


In [132]:
#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])
 
text_labels = encoder.classes_
print(text_labels)
 
for i in range(2):
    prediction = model.predict(np.array([x_test[i]]))
    print(np.argmin(prediction[0]))
    predicted_label = text_labels[np.argmin(prediction[0])]
    #print(test_files_names.iloc[i])
    print('Actual label:' + test_tags.iloc[i])
    print("Predicted label: " + predicted_label)

1240/1240 [==============================] - 0s 157us/step
Test accuracy: 0.9129032271523629
['bot' 'human']
0
Actual label:bot
Predicted label: bot
0
Actual label:bot
Predicted label: bot


In [206]:
#Evaluate model..on one test set at a time...........................
count_gender=0
count_type=0
for i in range(len(x_test)):
    text_labels = encoder.classes_
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmin(prediction[0])]
    print('Actual type:' + test_tags.iloc[i])
    print("Predicted type: " + predicted_label)
    if test_tags.iloc[i]==predicted_label:#good point to check acc for type
        count_type=count_type+1
        
    if predicted_label == "human":
        text_labels1 = encoder1.classes_
        prediction1 = model1.predict(np.array([x_test[i]]))
        predicted_label = text_labels1[np.argmin(prediction1[0])]
        print('Actual_1 gender.....:' + test_tags1.iloc[i])
        print("Predicted_1 gender: " + predicted_label)
    if predicted_label == test_tags1.iloc[i]:
        count_gender = count_gender+1
print("acc count_type = ",i, count_type, count_type/(i+1))
print("acc count_gender = ",i, count_gender, count_gender/(i+1))
        
        
   

len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted ty

Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: human
Actual_1 gender.....:bot
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:bot
Predicted type: human
Actual_1 gender.....:bot
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1


Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: human
Actual_1 gender.....:bot
Predicted_1 gender: female
len([x_test[i]

Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: bot
len([x_test[i]]== 1
Actual type:bot
Predicted type: human
Actual_1 gender.....:bot
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:bot
Predicted type: human
Actual_1 gender.....:bot
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:bot
Predicted type: human
Actual_1 gende

Actual_1 gender.....:male
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human

Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]==

Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:male
Predicted_1 gender: male
len([x_test[i]]

Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual 

Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human

Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: male
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender: female
len([x_test[i]]== 1
Actual type:human
Predicted type: human
Actual_1 gender.....:female
Predicted_1 gender

In [10]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
import itertools
from collections import Counter
##from readfile033 import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import time
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
#tt = ToktokTokenizer()#keeps the url as is...................................................
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
start = time. time()

B_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_BOT_train.2.txt"
H_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_HUMAN_train.2.txt"

B_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_BOT_dev.2.txt"
H_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_HUMAN_dev.2.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN_2019_txtfiles/PAN2019_tweet_male_train.2.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN_2019_txtfiles/PAN2019_tweet_female_train.2.txt"


def split_list(a_list):
        #function to split array
    half = len(a_list)/2
    print("half",half)
    return a_list[:half], a_list[half:]

def all_txt(B_txt_file,  H_txt_file):
    with open(B_txt_file, mode="r", encoding="utf-8") as f:
            n=100
##            [f[i:i+n] for i in range(0, len(f), n)]))
            txt_list = [(line[:-1]).split("\t")  for line in f]
##            txt_list= [txt_list[i:i+n] for i in range(0, len(txt_list), n)]
            bot_type= [line[2] for line in txt_list]
            
            bot_gender= [line[3] for line in txt_list]
           
            bot_content= [line[4] for line in txt_list]
##            print(bot_content[0])
            bot_content= [(bot_type[i],bot_gender[i], bot_content[i:i+n]) for i in range(0, len(bot_content), n)]
            print("bot_content....",len(bot_content))
##            bot_content= [("bot", (' '.join(a)).split()) for a in bot_content]
##            bot_content= [(' '.join(a)).split() for a in bot_content]
            
##            bot_dict = dict()
##            content=[]
##            for line in txt_list:
##                print(line[1])
##                if line[1] in bot_dict:
##                # append the new number to the existing array at this slot if they have the same id
##                    bot_dict[line[1]].append(line[3])
##                else:
##                # create a new array in this slot
##                    bot_dict[line[1]] = [line[3]]
##    bot_content= [("bot", (' '.join(a)).split()) for a in list(bot_dict.values())]
##    bot_content= [bot_content[i:i+n] for i in range(0, len(bot_content), n)]
   
    with open(H_txt_file, mode="r", encoding="utf-8") as f:
            n=100
            txt_list = [(line[:-1]).split("\t") for line in f]
##            txt_list= [txt_list[i:i+n] for i in range(0, len(txt_list), n)]
            human_type= [line[2] for line in txt_list]
            human_gender= [line[3] for line in txt_list]
            human_content= [line[4] for line in txt_list]
            human_content= [(human_type[i], human_gender[i],  human_content[i:i+n]) for i in range(0, len(human_content), n)]
            print("human_content....",len(human_content))
##            human_content= [(' '.join(a)).split() for a in human_content] [item for sublist in bot_content[i:i+n] for item in sublist]
##            human_dict = dict()
##            content=[]
##            for line in txt_list:
##                if line[1] in human_dict:
##                # append the new number to the existing array at this slot
##                    human_dict[line[1]].append(line[3])
##                else:
##                # create a new array in this slot
##                    human_dict[line[1]] = [line[3]]
##    human_content= [("human", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = bot_content + human_content
    return all_txt_per_person

def all_txt1(M_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            n=100
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_type= [line[2] for line in txt_list]
            male_gender= [line[3] for line in txt_list]
            male_content= [line[4] for line in txt_list]
            male_content= [(male_type[i], male_gender[i],   male_content[i:i+n] ) for i in range(0, len(male_content), n)]
##            male_content= [("human", (' '.join(a)).split()) for a in male_content]
##    with open(F_txt_file, mode="r", encoding="utf-8") as f:
##            txt_list = [(line[:-1]).split("\t") for line in f]
##            female_content= [(line[2], line[3]) for line in txt_list]
##    all_txt_per_person = male_content + female_content
    return male_content


"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):
## all_training_text = combined text from all authors, separated on per author basis
## word_list = list containing selected words(selected terms) to be used as a feature set ... the word list is saved in a csv/txt file which can be parsed as required

    #pos_list = ['NNP', 'PRP', 'DT', 'NN', 'CD', 'NNS', 'IN' , 'VB', 'PRP$']
    fvs_words = np.array([[author.count(word) for word in word_list]
                           for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words

def WordFeatures1(word_list, all_training_text):
    fvs_words = np.array([(all_training_text.count(word))/(len(all_training_text)) for word in word_list]).astype(np.float64)
    return fvs_words

"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
word_list11=[]
#txt_files =["D:/NLP/PAN2019/PAN2019/2019_selectterms_b_h/outputNewVersionbothuman2019CHI.txt"]
#txt_files =["D:/NLP/PAN2019/PAN2019/2019_selectterms_b_h/outputNewVersionbothuman2019WLLR.1M.txt", "D:/NLP/PAN2019/PAN2019/2019_selectterms_b_h/outputNewVersionbothuman2019WLLR.2F.txt"]
#txt_files =["D:/NLP/PAN2019/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_bot_nh_chosen_set.1.csv", "D:/NLP/PAN2019/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_human_chosen_set.1.csv"]
##x =(pd.read_excel('D:/NLP/PAN2019/PAN2019/PAN2019-Terms.xls', header=0, parse_cols = "C"))
##y =(pd.read_excel('D:/NLP/PAN2019/PAN2019/PAN2019-Terms.xls', header=0, parse_cols = "F"))
#word_list=list(x.Term) + list(y.Term)
##print( list(y.Term))
word_listf=[]
word_listB=[]
word_listH=[]
fntxt_files=["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/listFunctionWords.EN.txt"]
Btxt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/VocUniq.Bot.txt"]
Htxt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/VocUniq.Human.txt"]

txt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_bot_chosen_set.2.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv"]
txt_files1 =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv"]

##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2019/2019TermSelection/New folder/outputBotHuman2019IG.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019CHI.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019GR.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/2019TermSelection/outputBotHuman2019IG.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019GSS.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019OR.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019OR.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019PMI.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019PMI.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019RF.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019RF.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019WLLR.1M.txt", "D:/NLP/PAN_TermSelection/2019TermSelection/outputBotHuman2019WLLR.2F.txt"]##
#txt_files1 =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2019/terms_associated_to_Bot_or_Human/AVocUniq.Human.txt","/Users/catherine/Desktop/NLP/PAN Datasets/PAN2019/terms_associated_to_Bot_or_Human/AVocUniq.Bot.txt"]
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

for txt_file in txt_files1:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list2 =  [r[0] for r in reader]
        print(txt_file, len(word_list2))
        word_list11 = word_list11 + word_list2
print("length of features used =", len(word_list1))

#features for preclassification......................................................
for txt_file in Btxt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_listBB =  ([(r.split(":"))[1] for r in f])
        non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
        word_listBB = [(items.translate(non_bmp_map))[:-1] for items in word_listBB]
        word_listB = word_listB + word_listBB
##        print(txt_file, len(word_listB),word_listB)
        
for txt_file in Htxt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_listHH =  ([(r.split(":"))[1] for r in f])
        non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
        word_listHH = [(items.translate(non_bmp_map))[:-1] for items in word_listHH]
        word_listH = word_listH + word_listHH
        
for txt_file in fntxt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        fn_word_list =  ([r[:-1] for r in f])


##
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        word_list2 =  ([r.strip() for r in f])[0:100]
##        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))
##for txt_file in txt_files1:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        word_list1 =  ([(r.split(":"))[1] for r in f])
 #       non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
  #      word_list1 = ([(x.translate(non_bmp_map))[:-1] for x in word_list1])
   #     print(txt_file, len(word_list1))
    #    word_list = word_list + word_list1#
##print("length of features used =", len(word_list), (word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
Training_text= all_txt(B_train_file,  H_train_file)
##all_training_text = ( [((all_txt(B_train_file,  H_train_file))[i][2])  for i in range(len((all_txt(B_train_file,  H_train_file))))])
all_training_text = ( [((Training_text)[i][2])  for i in range(len(Training_text))] )
all_training_text=[(' '.join(a)).split() for a in all_training_text]
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
#print("shape...(all_training_text)",(X_train).shape)
##y_train=np.array( [((all_txt(B_train_file,  H_train_file))[i][0])  for i in range(len((all_txt(B_train_file,  H_train_file))))])
y_train=np.array( [((Training_text)[i][0])  for i in range(len(Training_text))])

print("len(y_train)=", (y_train.shape))
y_train11=np.array( [((Training_text)[i][1])  for i in range(len(Training_text))])
#print("len(X_train)=",(X_train.shape), "len(y_train11)=", len(y_train))


Test_text=all_txt(B_test_file,  H_test_file)
all_test_text = ( [((Test_text)[i][2])  for i in range(len(Test_text))])
all_test_text=[(' '.join(a)).split() for a in all_test_text]
print("len(all_test_text)",len(all_test_text))
#XX_test = WordFeatures(word_list, all_test_text)
yy_test = np.array( [((Test_text)[i][0])  for i in range(len(Test_text))])
print("len(y_test11)=",(yy_test.shape))
y_test11 = np.array( [((Test_text)[i][1])  for i in range(len(Test_text))])
print("len(y_test11=",len(y_test11), "len(y_test)=",len(yy_test))



"""
Parsing to the 2nd classifies............................................................
"""
Test_text1=all_txt1(H_train_file)
all_test_text1 = ( [((Test_text1)[i][2])  for i in range(len(Test_text1))])
all_test_text1=[(' '.join(a)).split() for a in all_test_text1]
print("mean length = ",(len([x for item in all_test_text1 for x in item]))/len(all_test_text1))
print("len(all_test_text)",len(all_test_text1))
#X_train1 = WordFeatures(word_list11, all_test_text1)
y_train1 = np.array( [((Test_text1)[i][1])  for i in range(len(Test_text1))])
print("y_train1...",y_train1)



#..................................................................................



/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_bot_chosen_set.2.csv 5602
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv 5828
length of features used = 11430
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/Pan2019_tweet_by_tweet/PAN2019_tweet_human_chosen_set.2.csv 5828
length of features used = 5828
bot_content.... 1440
human_content.... 1440
len(all_training_text) 2880
len(y_train)= (2880,)
bot_content.... 620
human_content.... 620
len(all_test_text) 1240
len(y_test11)= (1240,)
len(y_test11= 1240 len(y_test)= 1240
mean length =  2085.335416666667
len(all_test_text) 1440
y_train1... ['male' 'male' 'male' ... 'female' 'female' 'female']
